<a href="https://colab.research.google.com/github/BrianKani/GIS-CropForecast/blob/master/Data_landcover_entire_county.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geemap
import ee

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
import time
import numpy as np
import pandas as pd

In [ ]:

def export_oneimage(img, folder, name, region, scale, crs):
    task = ee.batch.Export.image.toDrive(image=img, description=name, folder=folder, fileNamePrefix=name, region=region, scale=scale, crs=crs)
    task.start()
    while task.status()['state'] == 'RUNNING':
        print('Running...')
        time.sleep(10)
    print('Done.', task.status())

locations = pd.read_csv('/content/locations.csv')

# Define Uasin Gishu County bounding box coordinates
uasin_gishu_bbox = [[34.6898, 0.3601], [35.8078, 0.3601], [35.8078, 0.9558], [34.6898, 0.9558]]

# Transforms an Image Collection with 1 band per Image into a single Image with items as bands
def appendBand(current, previous):
    # Rename the band
    previous = ee.Image(previous)
    current = current.select([0])
    # Append it to the result (Note: only return current item on first element/iteration)
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous, None), current, previous.addBands(ee.Image(current)))
    # Return the accumulation
    return accum

county_region = ee.FeatureCollection('ft:18Ayj5e7JxxtTPm1BdMnnzWbZMrxMB49eqGDTsaSp')

# Filter the image collection for your analysis period and area of interest
imgcoll = ee.ImageCollection('MODIS/051/MCD12Q1') \
    .filterBounds(ee.Geometry.Polygon(uasin_gishu_bbox)) \
    .filterDate('2016-12-31', '2022-12-31')
img = imgcoll.iterate(appendBand)
img = ee.Image(img)

for loc1, loc2, lat, lon in locations.values:
    fname = '{}_{}'.format(int(loc1), int(loc2))

    scale = 500
    crs = 'EPSG:4326'

    # Filter for Uasin Gishu County
    region = county_region.filterMetadata('STATE num', 'equals', loc1)
    region = ee.FeatureCollection(region).filterMetadata('COUNTY num', 'equals', loc2)
    region = region.first()
    region = region.geometry().bounds().getInfo()['coordinates']

    while True:
        try:
            export_oneimage(img, 'Data_county_mask', fname, region, scale, crs)
        except:
            print('Retry...')
            time.sleep(10)
            continue
        break
